In [13]:
import cv2
import numpy as np
import pandas as pd
import pickle as pkl
import imageio
from matplotlib import pyplot as plt
from optical_flow import *
from tqdm import tqdm
from tracking_utils import *
import matplotlib.animation as animation
import pickle as pkl
from load_utils import *
from eval_utils import *
from sort import *
from VehicleDetection import *

In [14]:
# SELECT SEQUENCE TO EVALUATE
S = 'seq3'

# SELECT CAMERA TO EVALUATE
C = 'c10'

# SET NUMBER OF FRAMES
num_frames = 2141

In [15]:
# Load GT detections
gt_detect_path = f'./cam_pred/{S}_{C}_gt.pkl'

# Load computed detections
detection_path = f'./cam_pred/{S}_{C}.pkl'

with open(detection_path, 'rb') as f:
    all_detections = pkl.load(f)

with open(gt_detect_path, 'rb') as f:
    all_gt_detections = pkl.load(f)


In [16]:
# Load video frames
results = {}

mot_tracker = Sort(max_age=10, min_hits=3, iou_threshold=0.3) # Sort Kalman tracker with default values

# Iterate Frames
for frame in tqdm(range(num_frames), desc = "Tracking objects each frame..."):
    dets = []

    detections = get_detection_dataframe(all_detections[str(frame)], iclLineAndUpdate = False, firstFrame = True).sort_values("track") # Load detections

    for (track_id, det, bbox, size, colour) in detections.itertuples(index=False): # Iter All Detections
        dets.append(np.array(bbox))


    trackers = mot_tracker.update(np.array(dets)) # Update tracker with current detections


    for d in trackers: # Store new bboxes
        d = d.astype(np.int32)
        if frame not in results:
            results[frame] = {d[4] % 47: {"bbox": [d[0], d[1], d[2], d[3]]}}
        else:
            results[frame][d[4] % 47] = {"bbox": [d[0], d[1], d[2], d[3]]}

            
# Save Results to Disk
pkl.dump(results, open(f"sort_bbox_{S}_{C}.pkl", "wb"))

Tracking objects each frame...: 100%|██████████| 2141/2141 [00:13<00:00, 162.69it/s]


In [17]:
# Pkl to pd.DataFrame
colours = np.random.rand(47, 3) #Generate Random Colors
old_pkl = pkl.load(open(f"sort_bbox_{S}_{C}.pkl", "rb"))
new_pkl = []
df = None

for frame in old_pkl.keys():
    new_data = {"frame": [],"track": [], "bbox": [], "colour": [], "size": [], "detection": []}
    old_data = old_pkl[frame]
    
    for track_id in old_data.keys():
        new_data["track"].append(track_id)
        bbox = old_data[track_id]["bbox"]
        new_data["bbox"].append(bbox)
        new_data["frame"].append("Frame {}".format(frame))
        new_data["colour"].append(np.round(colours[track_id]*255).astype(np.uint8))
        new_data["size"].append(int(np.abs(bbox[0] - bbox[2]) * np.abs(bbox[1] - bbox[3])))
        new_data["detection"].append(VehicleDetection(frame=frame, ID=track_id, width=np.abs(bbox[0] - bbox[2]), conf=0.5,
                                                      height=np.abs(bbox[1] - bbox[3]), left=bbox[0], right=bbox[2], top=bbox[1], bot=bbox[3]))
    
    df = pd.DataFrame.from_dict(new_data).sort_values(by="track")

    new_pkl.append(df)

pkl.dump(new_pkl, open(f"kalman_tracking_{S}_{C}.pkl", "wb"))

In [202]:
detections = {}
with open(f'kalman_tracking_{S}_{C}.pkl','rb') as openFile:
    detections = pkl.load(openFile)

## Evaluation

In [203]:
total_frames = num_frames
initial_frame = 1

In [204]:
i = initial_frame

while i < total_frames:
    detections[i]['parked'] = False
    for index_prev, previous_detection in detections[i-1].iterrows():
        length, _ = detections[i].shape
        for j in range(length):
            IoU = previous_detection['detection'].IoU(detections[i].iloc[j]['detection'])
            if IoU > 0.95:
                detections[i].at[j, 'parked'] = True
    i += 1

In [201]:
acc = create_accumulator()

# detections = detections[detections['parked'] == False]

for frame in tqdm(range(initial_frame, total_frames-1)):
    detections[frame] = detections[frame][detections[frame]['parked'] == False]
    if str(frame) in all_gt_detections:
        acc, frame_id = tracking_acc(frame, all_gt_detections, detections, acc)

print(f'{S} {C} results:')
display_metrics(acc)

100%|██████████| 2139/2139 [00:15<00:00, 139.34it/s]

seq3 c10 results:
     num_frames  precision    recall       idp      idr      idf1
acc        1355   0.367835  0.944984  0.184548  0.47411  0.265679
